In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from fuzzywuzzy import process, fuzz

In [2]:
from counting.szp_funcs import to_double, matcher, load_jobs, load_pay_types, print_df

In [3]:
job_106 = load_jobs()
pay_types = load_pay_types()

In [4]:
look_up_jobs = job_106['pre_job'].unique().tolist()
look_up_pay = pay_types['name'].unique().tolist()

In [5]:
dir_names = []
path_to_dirt = 'C:\\Users\\PetukhovMD\\Desktop\\2021\\'

In [6]:
import os

for file in os.listdir(path_to_dirt):
    d = os.path.join(path_to_dirt, file)
    if os.path.isdir(d):
        dir_names.append(d)

In [7]:
dir_names

['C:\\Users\\PetukhovMD\\Desktop\\2021\\Август',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Апрель',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Декабрь',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Июль',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Июнь',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Май',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Март',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Ноябрь',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Октябрь',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Сентрябрь',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Февраль',
 'C:\\Users\\PetukhovMD\\Desktop\\2021\\Январь']

In [8]:
months = ['aug', 'apr', 'dec', 'jul', 'jun', 'may', 'mar', 'nov', 'oct', 'sep', 'feb', 'jan']
post_path = 'C:\\Users\\PetukhovMD\\Desktop\\szp_2021\\'

In [9]:
nach_name = 'ДОНМ_Начисления работников.xlsx'
pay_name = 'ДОНМ_Начисления работников по категориям.xlsx'
kadr_name = 'ДОНМ_Кадровые_паспортные данные.xlsx'

In [10]:
def give_day(row):
    if pd.isna(row['wday']) | pd.isna(row['nday']):
        return 0
    if row['wday'] >= row['nday']:
        return 1
    return 0

In [11]:
def give_sum(row):
    if row['type'] == 'Основное место работы':
        return row['sum_curr'] + row['sum_prev']
    return row['sum_curr']

In [13]:
for i in range(len(dir_names)):
    nach = pd.read_excel(dir_names[i] + '\\' + nach_name)
    clean_nach = nach.iloc[:, [2, 4, 8, 9, 10, 11, 12, 13, 17, 19, 20]]
    clean_nach.columns = ['inn', 'snils', 'job_dirty', 'type', 'stv', 'status', 'sum_all', 'sum', 'status_pref', 'wday', 'nday']
    clean_nach['stv'] = clean_nach.apply(lambda row: to_double(row, 'stv'), axis=1)
    clean_nach['day'] = clean_nach.apply(lambda row: give_day(row), axis=1)
    cleaning_jobs = pd.DataFrame(clean_nach['job_dirty'].unique().tolist()).rename(columns={0: 'job_dirty'})
    cleaning_jobs['pre_job'] = cleaning_jobs.apply(lambda row: matcher(row, look_up_jobs, 'job_dirty'), axis=1)
    cleaning_jobs = pd.merge(cleaning_jobs, job_106, how='left').drop(columns=['pre_job']).rename(columns={'job_106': 'job'}).drop_duplicates('job_dirty')
    print_df(cleaning_jobs, post_path + 'vlookup_jobs_' + months[i])
    #print vlookup_jobs here
    clean_nach = pd.merge(clean_nach, cleaning_jobs, how='left').drop(columns=['job_dirty'])
    print_df(clean_nach, post_path + months[i])
    #print nach here month.xlsx
    pay = pd.read_excel(dir_names[i] + '\\' + pay_name)
    pay_clean = pay.iloc[:, [2, 3, 6, 10, 13, 14]]
    pay_clean.columns = ['inn', 'snils', 'job_dirty', 'type', 'pre_name', 'nach_sum']
    pay_clean = pd.merge(pay_clean, cleaning_jobs, how='left').drop(columns=['job_dirty'])
    print_df(pay_clean, post_path + months[i] + '_nach')
    #print pay here month_nach.xlsx
    kadr = pd.read_excel(dir_names[i] + '\\' + kadr_name)
    kadr_clean = kadr.iloc[:, [2, 3, 7, 8, 11]]
    kadr_clean.columns = ['inn', 'snils', 'job_dirty', 'type', 'status']
    kadr_clean = pd.merge(kadr_clean, cleaning_jobs, how='left').drop(columns=['job_dirty'])
    print_df(kadr_clean, post_path + months[i] + '_kadr')
    #print kadr here month_kadr.xlsx

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_29840\3644821581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_nach['stv'] = clean_nach.apply(lambda row: to_double(row, 'stv'), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_29840\3644821581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_nach['day'] = clean_nach.apply(lambda row: give_day(row), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_29840\3644821581.py:5: SettingWithCopyWarning: 
A value is trying to be set